In [2]:
from fastembed import TextEmbedding

c:\Users\marvi\myvenv\envs\llmzoomcamp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Q1

In [3]:
model_handle = TextEmbedding('jinaai/jina-embeddings-v2-small-en')
results = model_handle.embed('I just discovered the course. Can I join now?')
embedding = next(results)
print(min(embedding))

-0.11726373885183883


Q2

In [4]:
doc = 'Can I still join the course after the start date?'
results = model_handle.embed(doc)
embedding2 = next(results)
cosine_similarity = embedding.dot(embedding2)
print(cosine_similarity)

0.9008528895674548


Q3

In [5]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [9]:
import numpy as np

texts = [doc['text'] for doc in documents]
results = model_handle.embed(texts)
text_embedding_matrix = []
for result in results:
    text_embedding_matrix.append(result)
text_embedding_matrix = np.array(text_embedding_matrix)    

[[-0.0249525  -0.0396454  -0.00437672 ...  0.06779866  0.0377203
  -0.00470724]
 [-0.0594709  -0.08523984  0.01292091 ...  0.09599709  0.05420843
  -0.00029461]
 [-0.06846454 -0.04079098  0.04998121 ...  0.06578071  0.02872077
  -0.01115215]
 [-0.04640506 -0.02555229  0.02241224 ...  0.03410312  0.06813592
  -0.0064033 ]
 [-0.05394912 -0.04693814  0.00794725 ...  0.05402997  0.03033385
  -0.01254518]]


In [12]:
score = embedding @ text_embedding_matrix.T
print(score)

[0.76296847 0.81823782 0.80853974 0.7133079  0.73044992]


Q4

In [13]:
full_text = [doc['question'] + ' ' + doc['text'] for doc in documents]
results = model_handle.embed(full_text)
text_embedding_matrix = []
for result in results:
    text_embedding_matrix.append(result)
text_embedding_matrix = np.array(text_embedding_matrix)   

In [14]:
score = embedding @ text_embedding_matrix.T
print(score)

[0.85145432 0.84365942 0.8408287  0.7755158  0.80860078]


In [16]:
supported_models = TextEmbedding.list_supported_models()
min_model_index = 0
min_model_dim = 512
for i in range(len(supported_models)):
    if supported_models[i]['dim'] < min_model_dim:
        min_model_index = i
        min_model_dim = supported_models[i]['dim']
print(supported_models[min_model_index])

{'model': 'BAAI/bge-small-en', 'sources': {'hf': 'Qdrant/bge-small-en', 'url': 'https://storage.googleapis.com/qdrant-fastembed/BAAI-bge-small-en.tar.gz', '_deprecated_tar_struct': True}, 'model_file': 'model_optimized.onnx', 'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.', 'license': 'mit', 'size_in_GB': 0.13, 'additional_files': [], 'dim': 384, 'tasks': {}}


Q6

In [20]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [21]:
from qdrant_client import QdrantClient, models
client = QdrantClient("http://localhost:6333")

In [22]:
collection_name="02-hw"

# Create the collection with specified sparse vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=384,
        distance=models.Distance.COSINE
    )
)

True

In [25]:
points = []
id = 0

for doc in documents:
    text = doc['question'] + ' ' + doc['text']
    point = models.PointStruct(
        id=id,
        vector=models.Document(text=text, model='BAAI/bge-small-en'),
        payload={
            "text": doc['text'],
            "section": doc['section'],
            "course": course['course']
        }
    )
    points.append(point)
    id += 1

In [26]:
client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]c:\Users\marvi\myvenv\envs\llmzoomcamp\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\marvi\AppData\Local\Temp\fastembed_cache\models--Qdrant--bge-small-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Fetching 5 files: 100%|██████████| 5/5 [00:

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [29]:
results = client.query_points(
    collection_name=collection_name,
    query=models.Document(text='I just discovered the course. Can I join now?', model='BAAI/bge-small-en'),
    limit=1,
    with_payload=True
)

In [34]:
results.points[0].score

0.8703172